In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks

from pydub.silence import detect_nonsilent
import pandas as pd

import librosa
import matplotlib.pyplot as plt
import librosa.display

import numpy as np

# import noisereduce as nr


In [ ]:
# Use this script to test and find the best silence lens and threshold
# Splices fragment of audio file
#TODO https://stackoverflow.com/questions/59102171/getting-timestamps-from-audio-using-python
#TODO add directory


#Input directory example
#/Users/michael/Desktop/test_demo.m4a

# guide
# target_length: If chunk length < than 5 secs, it will attempt to pad the chunk by retaining natural silence
# min_silence_len: Determine how long silences/pauses are too long and unneccessary 
# Silence_thresh: anything under the defined dBFS is considered silence. The lower the silence threshold, the less sensitive it can pick silence


#Parameters
seconds = 1000 # 1000 ms
predefined_ms = 60 * seconds
target_dBFS = -20.0
min_silence_len = 4 * 1000
target_silence_thresh = -38
channels = 1 # global variable
frame_rate = 22050 # global variable
seek_step = 1

audio_path = input("Input audio path") #path refers to the file
loaded_audio = AudioSegment.from_file(audio_path, format="m4a")

#Splices a section of the audio for testing
# spliced_test_audio = loaded_audio[:predefined_ms] #first 60 seconds
# spliced_test_audio = loaded_audio[(predefined_ms*1):(predefined_ms * 2)] # second 60 seconds
# spliced_test_audio = loaded_audio[(predefined_ms *2):(predefined_ms * 3)]  # third 60 seconds
spliced_test_audio = loaded_audio[(predefined_ms *3):(predefined_ms * 4)]  # fourth 60 seconds


#Splices the last 60 seconds of the audio
# spliced_test_audio = loaded_audio[-predefined_ms:] 

#change to monochannel and change sampling rate
spliced_test_audio  = spliced_test_audio.set_channels(channels)
spliced_test_audio = spliced_test_audio.set_frame_rate(frame_rate)

#Export spliced sample audio
spliced_test_audio.export("test_sample.wav", format="wav")# Use this script to test and find the best 
print("Audio successfully spliced")


In [ ]:
#Use this to adjust parameters. Check if the start and stop times correspond with wave plot.

#Convert test wav to audio_segment
audio_segment = AudioSegment.from_wav("test_sample.wav")

#Convert seconds into timestamp
def convert(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%d:%02d:%02d" % (hour, minutes, seconds) 

#adjust target amplitude
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

#normalize audio_segment to -20dBFS 
normalized_sound = match_target_amplitude(audio_segment, target_dBFS)
audio_length = len(normalized_sound)/1000 
print("length of audio_segment={} seconds".format(audio_length))

#Export and display normalized amplitude file
normalized_sound.export("test_sample_normalized.wav", format="wav")

#Print detected non-silent chunks, which in our case would be spoken words.
#the lower the silence threshold (higher negative number), the less senstive it is
nonsilent_data = detect_nonsilent(normalized_sound, min_silence_len=min_silence_len, silence_thresh=target_silence_thresh, seek_step=seek_step)

#convert ms to seconds
print("start,Stop")
for chunks in nonsilent_data:
    print([convert(chunk/1000) for chunk in chunks])
 
 

  


In [ ]:
#Graphs the sample audio. Used to see if the silence parameters are acceptable
#ALso use

#Plots the sample test audio
test_dir = "test_sample.wav"
audio_data, sampling_rate = librosa.load(test_dir)
plt.figure(figsize=(14, 5))
plt.xticks(np.arange(min(audio_data), max(audio_data)+1, 1.0))
librosa.display.waveplot(audio_data, sr=sampling_rate)

#Plots the normalized test audio
test_dir = "test_sample_normalized.wav"
audio_data, sampling_rate = librosa.load(test_dir)
plt.figure(figsize=(14, 5))
librosa.display.waveplot(audio_data, sr=sampling_rate)



In [ ]:
# # Reduces noise and graphs the sample. Compare this graph above to see if its acceptable.

# #Determine which part of the audio can be used as a reference for background noise
# # done in ms / 1000m = 1 second
# noisy_part = audio_data[22000:26000]
# # perform noise reduction
# reduced_noise = nr.reduce_noise(audio_clip=audio_data,
# noise_clip=noisy_part, verbose=True)

# plt.figure(figsize=(14, 5))
# librosa.display.waveplot(audio_data, sr=sampling_rate)


